In [2]:
from parsing import *
from NJ import *
import os
import time
import pandas as pd

In [33]:
executions = 'unique_distance_matrices\execution_times.csv'
our_algorithm = 'execution_times_first5.xlsx'
df = pd.read_csv(executions)
our = pd.read_excel(our_algorithm)
our['file'] = our.iteration.str.split(pat="/",expand=True)[1]
## merged results
executions = pd.merge(our,df,how='right')
executions = executions[['file','QuickTreeNJ','RapidNJ','time']].fillna(0)
executions

,file,QuickTreeNJ,RapidNJ,time
0,1347_FAINT.phy,1.963585,0.465415,0.000000
1,1493_Fe-ADH.phy,2.282779,0.531238,0.000000
2,1560_Ferritin.phy,2.524688,0.543163,0.000000
3,1689_FGGY_N.phy,3.411204,0.680182,0.000000
4,1756_FAD_binding_3.phy,3.475785,0.855618,0.000000
5,1849_FG-GAP.phy,3.998725,0.797029,0.000000
6,214_Arena_glycoprot.phy,0.106196,0.026261,239.704620
7,304_A1_Propeptide.phy,0.157629,0.045671,964.513976
8,401_DDE.phy,0.220496,0.054480,4475.419575
9,494_Astro_capsid.phy,0.303667,0.064701,7221.924666
